## Aug 18, 2024 - Plotly Figure Friday - Week 33 - US Presidential Elections
Analyze results over time, this analysis is not map based
- plot winning party of each state over all years in the data set, like D = 1, R= 0
- plot change in DEM_PCT by state per year, but subtraction of value in first year.

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import numpy as np

#------------------------------------------------------------------------------#
#     Read dataset from local file, clean up - uses polars LazyFrame           #
#------------------------------------------------------------------------------#
keep_columns = [
    'candidate', 'year', 'state', 'state_po', 
    'party_simplified', 'candidatevotes','totalvotes'
]
df = (
    pl.scan_csv(       # pl.scan_csv creates Lazy Frame for query optimization
    './Dataset/1976-2020-president.csv',
    ignore_errors=True
    )
    .select(
        pl.col(keep_columns))
    .filter(pl.col('party_simplified').is_in(['DEMOCRAT', 'REPUBLICAN']))
    .with_columns(
        pl.col('party_simplified')
        .str.replace('DEMOCRAT', 'DEM')
        .str.replace('REPUBLICAN', 'REP')
    )
    .filter(pl.col('candidate') != 'OTHER')
    .with_columns(
        pl.col('candidate')
            # for unique candidate family names, fix the Bushes
            .str.replace('BUSH, GEORGE H.W.', 'Bush Senior')
            .str.replace('BUSH, GEORGE W.', 'Bush Junior')
            # for unique candidate family names, fix the Clintons
            .str.replace('CLINTON, BILL', 'Clinton B.')
            .str.replace('CLINTON, HILLARY', 'Clinton H.')
            # ROMNEY, MITT wrongly listed as MITT ROMNEY, Washington 2012
            .str.replace('MITT, ROMNEY', 'ROMNEY, MITT'),
        pl.col('year').cast(pl.Int16)
    )
    .with_columns(             # get last name of candidate
        pl.col('candidate')
        .str.split(',')        
        # .list.slice(0, 1)      # gets all text up to first comma
        .list.first()           # gets all text up to first comma, usually the last name
        .str.to_titlecase()     # Title case breaks on McCain, next line fixes is
        .str.replace('Mccain', 'McCain')
    )
    # Un-named DEMOCRATIC candidate from ARIZONA in 2016 won 4 votes, drop nulls to remove
    .drop_nulls('candidate')
    .collect()    # collect applies auto-optimization to covert Lazy Frame to usable Data Frame
)

#------------------------------------------------------------------------------#
#     Make a list of candidate last names by election year. Special attention  #
#     for the Bushes (Junior & Senior) & the Clintons (Bill & Hillary)         #
#------------------------------------------------------------------------------#
df_candidates = (
    df
    .select(pl.col('candidate', 'year','party_simplified'))
    .unique()
    .sort('year')
    .pivot(index='year', on='party_simplified', values='candidate')
)

#------------------------------------------------------------------------------#
#     Group by year, state, calc difference between DEM_VOTES and REP_VOTES    #
#------------------------------------------------------------------------------#
df_votes = (
    df
    .rename({'totalvotes':'TOT_VOTES'})
    .pivot(
        index=['year', 'state', 'state_po','TOT_VOTES'],  # , 'candidate'], 
        on ='party_simplified', 
        values='candidatevotes', 
        aggregate_function='sum'
        )
    .with_columns(pl.col('year').cast(pl.Int16))
    .rename({'DEM': 'DEM_VOTES', 'REP': 'REP_VOTES'})
)

#------------------------------------------------------------------------------#
#     Use a lazy frame to goin df_votes and df_candidates, and finish          #
#     Remaining calculations                                                   #
#------------------------------------------------------------------------------#
final_df = (
    pl.LazyFrame(
        df_candidates.join(
            df_votes,
            on='year',
            how='left'
        )
        .with_columns(
            DEM_PCT = pl.col('DEM_VOTES') / pl.col('TOT_VOTES'),
            REP_PCT = pl.col('REP_VOTES') / pl.col('TOT_VOTES'),
        )
        .with_columns(   # DIFF_PCT is used for coloring each state
            DIFF_PCT = pl.col('DEM_PCT') -  pl.col('REP_PCT'),
        )
        .with_columns(   # absolute value of vote margin used by hovertemplate
            ABS_DIFF_PCT = abs(pl.col('DIFF_PCT'))
        )
        # make columns with winner's name and party for hovertemplate
        .with_columns(
            STATE_WINNER = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then('DEM')
                .otherwise('REP'),
            WINNING_PARTY = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then(pl.lit('D'))
                .otherwise(pl.lit('R'))
        )
        # make columns with loser's name and party for hovertemplate
        .with_columns(
            STATE_LOSER = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then('REP')
                .otherwise('DEM'),
            LOSING_PARTY = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then(pl.lit('R'))
                .otherwise(pl.lit('D'))
        )
         # make column with winers's vote percentage for hovertemplate
        .with_columns(
            WINNING_PCT = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then('DEM_PCT')
                .otherwise('REP_PCT')
        )
         # make column with losers's vote percentage for hovertemplate
        .with_columns(
            LOSING_PCT = 
                pl.when(pl.col('DEM_PCT')> pl.col('REP_PCT'))
                .then('REP_PCT')
                .otherwise('DEM_PCT')
        )
    )
    .collect()
)

#------------------------------------------------------------------------------#
#     Make dataframe  with year + states as columns. data shows winning party  #
#------------------------------------------------------------------------------#
df_state_winners = (
    final_df
    .select('year', 'state', 'state_po', 'WINNING_PARTY')
    .with_columns(
        pl.col('WINNING_PARTY').str.replace('R', 'REP').str.replace('D', 'DEM')
    )
    .pivot(
        index='year',
        on = 'state_po',
        values='WINNING_PARTY'
    )
    .with_columns(pl.col('year').cast(pl.String))
    .with_columns(pl.all().cast(pl.String))
)

#------------------------------------------------------------------------------#
#     Make scatter plot of each state's winning party by year                  #
#------------------------------------------------------------------------------#
states = df_state_winners.columns[1:]
fig = px.scatter(df_state_winners, 'year', states)
fig.update_layout(
    title = f'USA Elections: State Winner by year'.upper(),
    height=300, width=800,
    xaxis_title='ELECTION YEAR'.upper(),
    yaxis_title="Winning party".upper(),
    yaxis_title_font=dict(size=14),
    xaxis_title_font=dict(size=14),
    margin={"r":50, "t":50, "l":50, "b":50},
    autosize=False,
    showlegend=True,
    template='plotly_white'
)
#  Setup hover elements
fig.update_traces(
    mode='markers+lines',
    marker=dict(size=12, line=dict(width=0)),
    )
fig.update_layout(hovermode='x unified')
fig.update_layout(legend_title='STATE')
fig.update_yaxes(range = ['REP', 'DEM'])
fig.update_yaxes(categoryorder='category descending')

fig.show()

In [3]:
final_df.sample(5)

year,DEM,REP,state,state_po,TOT_VOTES,DEM_VOTES,REP_VOTES,DEM_PCT,REP_PCT,DIFF_PCT,ABS_DIFF_PCT,STATE_WINNER,WINNING_PARTY,STATE_LOSER,LOSING_PARTY,WINNING_PCT,LOSING_PCT
i16,str,str,str,str,i64,i64,i64,f64,f64,f64,f64,str,str,str,str,f64,f64
1976,"""Carter""","""Ford""","""MISSOURI""","""MO""",1953600,998387,927443,0.51105,0.474735,0.036314,0.036314,"""Carter""","""D""","""Ford""","""R""",0.51105,0.474735
1984,"""Mondale""","""Reagan""","""DELAWARE""","""DE""",254572,101656,152190,0.399321,0.597827,-0.198506,0.198506,"""Reagan""","""R""","""Mondale""","""D""",0.597827,0.399321
1984,"""Mondale""","""Reagan""","""DISTRICT OF COLUMBIA""","""DC""",211288,180408,29009,0.853849,0.137296,0.716553,0.716553,"""Mondale""","""D""","""Reagan""","""R""",0.853849,0.137296
2000,"""Gore""","""Bush Junior""","""ARKANSAS""","""AR""",921781,422768,472940,0.458643,0.513072,-0.054429,0.054429,"""Bush Junior""","""R""","""Gore""","""D""",0.513072,0.458643
2016,"""Clinton H.""","""Trump""","""VIRGINIA""","""VA""",3982752,1981473,1769443,0.497514,0.444276,0.053237,0.053237,"""Clinton H.""","""D""","""Trump""","""R""",0.497514,0.444276
